Importing relevant functionalities

In [ ]:
import re
import numpy as np
import scipy as sc
import matplotlib.pyplot as plt
import pandas as pd
from pandas import DataFrame as df
from sklearn.metrics import r2_score

In [ ]:
mols_g16_all = []
for i in range(33):
    i1 = ("%06d" % (i+1))
    i1 = str(i1)
    mols_g16_all.append(i1+'.log')
b3lyp = [] 
blyp = []
cam_b3lyp = []
lc_blyp = []
lc_whpbe = [] 
m062x = [] 
m11 = [] 
pbe1pbe = [] 
pbepbe = [] 
pw91pw91 = [] 
svwn5 = [] 
tpssh = [] 
tpsstpss = []
wb97xd = [] 
functionals_list = [b3lyp, blyp, cam_b3lyp, lc_blyp, lc_whpbe, m062x, m11, pbe1pbe, pbepbe, pw91pw91, svwn5, tpssh,
                   tpsstpss, wb97xd]
def listtostring(s1):
    str1 = ""
    for ele in s1:
        str1 += ele
    return str1
def namestr(obj, namespace):
    return [name for name in namespace if namespace[name] is obj]
for i in range(len(functionals_list)):
    for j in range(len(mols_g16_all)):
        s = namestr(functionals_list[i], globals())
        t = listtostring(s)
        functionals_list[i].append('../33_benchmark/'+t+'/'+mols_g16_all[j])

In [ ]:
functionals_list_v = [blyp, pbepbe, pw91pw91, svwn5, tpsstpss]
for i in range(len(functionals_list_v)):
    print(functionals_list_v[i][11])
    functionals_list_v[i].pop(11)
print(len(blyp),len(pbe1pbe))

Preparing csv files

In [ ]:
comp_pattern='Normal termination of Gaussian 16'
E1_pattern='Excited State   1:'
E2_pattern='Excited State   2:'
p1 = 4
p2 = 8
def extract_E12_f12_g16(filename):
    file = open(filename, "r")
    iline=0
    for line in file:
        if re.search(comp_pattern, line):
            iline=iline+1
    if (iline < 1):
        print('ERROR: Can\'t find <',comp_pattern,'> in ',filename)
        file.close()
    elif (iline==1):
        #print('Found <',comp_pattern,'> in ',filename)
        file.close()
        file = open(filename, "r")
        mark_e1 = 0
        for l1 in file:
            if re.search(E1_pattern, l1):
                mark_e1 = l1
                #print(mark_e1)
        file.close()
        file = open(filename, "r")
        mark_e2 = 0
        for l2 in file:
            if re.search(E2_pattern, l2):
                mark_e2 = l2
                #print(mark_e2)
        file.close()        
    E1 = float( mark_e1.split()[p1] ) 
    f0 = str( mark_e1.split()[p2] ) 
    fx = float(re.sub("[^0123456789\.]", '', f0))
    f1 = float("{:.6f}".format(fx))
    E2 = float( mark_e2.split()[p1] ) 
    f0 = str( mark_e2.split()[p2] ) 
    fx = float(re.sub("[^0123456789\.]", '', f0))
    f2 = float("{:.6f}".format(fx))
    return E1, f1, E2, f2
def make_csv(filename,Nrows,Ncols,Header,Values):
    file = open(filename, "w")
    for icol in range(Ncols):
        if icol == Ncols-1:
            file.write(Header[icol]+'\n')
        else:
            file.write(Header[icol]+',')
    for irow in range(Nrows):
        for icol in range(Ncols):
            if icol == Ncols-1:
                file.write(str(Values[icol][irow])+'\n')
            else:
                file.write(str(Values[icol][irow])+',')
    file.close()   
    return 1

In [ ]:
H1 = ['title', 'E1', 'f1', 'E2', 'f2']
for i in range(len(functionals_list)):
    V = np.zeros([5,len(functionals_list[i])])
    for imol in range(len(functionals_list[i])):
        mol = functionals_list[i][imol]
        E1, f1, E2, f2 =extract_E12_f12_g16(mol)
        V[0,imol] = imol + 1
        V[1,imol] = E1
        V[2,imol] = f1
        V[3,imol] = E2
        V[4,imol] = f2
        s = namestr(functionals_list[i], globals())
        t = listtostring(s)
    tmp = make_csv('../csv_files/'+t+'.csv',len(functionals_list[i]),len(H1),H1,V)